In [2]:
import torch
import torchaudio
import librosa
import random
import os
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torchaudio.functional as F
import torchaudio.transforms as T
import numpy as np
import pandas as pd
import scipy
from IPython.display import display, Audio

In [7]:
'''
1. We will first have to create a table for the available datasets and what they offer
2. We then choose the best datasets to work with try to integrate them into the already written code
3. We add bg noise second wise, choose a simple AE architecture and exclusively train it for a few days.

'''


OrderedDict([('encoder.0.weight',
              tensor([[[-0.4919, -0.0245,  0.3662]],
              
                      [[-0.5609, -0.6425, -0.0044]],
              
                      [[-0.4187,  0.3025,  0.4607]],
              
                      [[-0.4099, -0.5097, -0.0335]],
              
                      [[ 0.4355, -0.0647,  0.0063]],
              
                      [[-0.2091, -0.2654, -0.1730]],
              
                      [[ 0.2453, -0.5539,  0.4524]],
              
                      [[-0.1437, -0.1989,  0.2833]],
              
                      [[ 0.5442, -0.2077, -0.1181]],
              
                      [[ 0.2471, -0.0788, -0.3006]],
              
                      [[-0.5486, -0.4049,  0.0734]],
              
                      [[ 0.3672,  0.5505, -0.2801]],
              
                      [[-0.1271, -0.2253,  0.2947]],
              
                      [[-0.4179,  0.6223, -0.5586]],
              
              